## Train weighted LR model on t2dv2 data
- Features used in this model: 'aligned_pagerank', 'smallest_qnode_number', 'monge_elkan', 'des_cont_jaccard_normalized'
- Trained model should be saved on disk for future load
- This part is not included in the actual table linker pipeline

## Dataset 
- Train: https://drive.google.com/drive/u/2/folders/1bUDGEI4gNEw6VY4Bquy0zy10ga-7Mb2M 
- Dev: https://drive.google.com/drive/u/2/folders/1RgxmFI7QGVh53Y_2OwvBk1Uyzu0Y2ftX

In [1]:
import numpy as np
import pandas as pd

In [23]:
HOME_DIR = '/Users/summ7t/dev/novartis/table-linker'
training_candidates_dir = f'{HOME_DIR}/t2dv2-train-candidates-dwd-v2'
training_features_dir = f'{HOME_DIR}/t2dv2-train-score/votes'
trained_model_file = f'{HOME_DIR}/weighted_lr.pkl'

dev_features_dir = f'{HOME_DIR}/t2dv2-dev-score/votes/'

In [3]:
# helper method we use
def clean_dataset(df):
    print(f"length of df: {len(df)}")
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    # df.dropna(inplace=True)
    print(f"length of df: {len(df)} after dropping na")
    indices_to_keep = ~df.loc[:, ['aligned_pagerank', 'smallest_qnode_number', 'monge_elkan', 'des_cont_jaccard_normalized']].isin([np.nan, np.inf, -np.inf]).any(1)
    print(f"length of indices_to_keep: {sum(indices_to_keep)}")
    return df[indices_to_keep]

# merge all eval files in one df
def merge_df(file_names: list):
    df_list = []
    for fn in file_names:
        fid = fn.split('/')[-1].split('.csv')[0]
        df = pd.read_csv(fn)
        df['table_id'] = fid
        # df = df.fillna('')
        df_list.append(df)
    
    return pd.concat(df_list)

#### Generate training data with 4 required features

In [4]:
!which tl

/Users/summ7t/dev/novartis/novartis_env/bin/tl


##### Script I used for generating vote files - voting_test_pipline.sh
```
filename=$1

tl smallest-qnode-number /Users/summ7t/dev/novartis/table-linker/t2dv2-dev-candidates-dwd-v2/$filename.csv \
/ align-page-rank \
/ string-similarity -i --method symmetric_monge_elkan:tokenizer=word -o monge_elkan \
/ string-similarity -i --method jaccard:tokenizer=word -c kg_descriptions context -o des_cont_jaccard \
/ normalize-scores -c des_cont_jaccard \
> /Users/summ7t/dev/novartis/table-linker/t2dv2-dev-score/votes/$filename.csv
```

In [7]:
# List all train dataset files
import os

file_names = []
file_ids = []

for (dirpath, dirnames, filenames) in os.walk(training_candidates_dir):
    for fn in filenames:
        if "csv" not in fn:
            continue
        abs_fn = os.path.join(dirpath, fn)
        assert os.path.isfile(abs_fn)
        if os.path.getsize(abs_fn) == 0:
            continue
        file_names.append(abs_fn)
        file_ids.append(fn.split('.csv')[0])
len(file_names), file_ids[:3]

(44,
 ['58891288_0_1117541047012405958',
  '39173938_0_7916056990138658530',
  '10579449_0_1681126353774891032'])

In [17]:
for idx, fid in enumerate(file_ids):
    print(f"generating training feature for {idx}th file: {fid}")
    training_candidate_f = f'{training_candidates_dir}/{fid}.csv'
    training_feature_f = f'{training_features_dir}/{fid}.csv'
    script = f"""
    tl smallest-qnode-number {training_candidate_f} \
    / align-page-rank \
    / string-similarity -i --method symmetric_monge_elkan:tokenizer=word -o monge_elkan \
    / string-similarity -i --method jaccard:tokenizer=word -c kg_descriptions context -o des_cont_jaccard \
    / normalize-scores -c des_cont_jaccard \
    > {training_feature_f}
    """
    os.system(script)
    assert os.path.isfile(training_feature_f), f"Something wrong with training feature result: {idx}th file: {fid}"

generating training feature for 0th file: 58891288_0_1117541047012405958
generating training feature for 1th file: 39173938_0_7916056990138658530
generating training feature for 2th file: 10579449_0_1681126353774891032
generating training feature for 3th file: 33401079_0_9127583903019856402
generating training feature for 4th file: 21362676_0_6854186738074119688
generating training feature for 5th file: 38428277_0_1311643810102462607
generating training feature for 6th file: 91959037_0_7907661684242014480
generating training feature for 7th file: 20135078_0_7570343137119682530
generating training feature for 8th file: 35188621_0_6058553107571275232
generating training feature for 9th file: 54719588_0_8417197176086756912
generating training feature for 10th file: 21245481_0_8730460088443117515
generating training feature for 11th file: 71840765_0_6664391841933033844
generating training feature for 12th file: 8468806_0_4382447409703007384
generating training feature for 13th file: 885233

In [19]:
fid = '58891288_0_1117541047012405958'
pd.read_csv(f'{training_features_dir}/{fid}.csv')

,column,row,label,context,label_clean,kg_id,kg_labels,kg_aliases,method,kg_descriptions,...,smallest_qnode_number,aligned_pagerank,monge_elkan.1,des_cont_jaccard.1,des_cont_jaccard_normalized,vote_aligned_pagerank,vote_smallest_qnode_number,vote_monge_elkan,vote_des_cont_jaccard,votes
0,1,0,The Godfather,1|1972|Francis Ford Coppola|1,The Godfather,Q13033687,The Godfather,NaN,exact-match,Grimm,...,3.659977e-03,7.119850e-09,1.000000,0.000,0.00,0,0,1,0,1
1,1,0,The Godfather,1|1972|Francis Ford Coppola|1,The Godfather,Q1158135,The Godfather,NaN,exact-match,soundtrack of the 1972 crime film of the same ...,...,4.118950e-02,5.141773e-09,1.000000,0.125,0.25,0,0,1,0,1
2,1,0,The Godfather,1|1972|Francis Ford Coppola|1,The Godfather,Q1139696,The Godfather,The Godfather: The Game|The Godfather: Mob War...,exact-match,2006 open world action-adventure video game,...,4.185590e-02,5.155339e-09,1.000000,0.000,0.00,0,0,1,0,1
3,1,0,The Godfather,1|1972|Francis Ford Coppola|1,The Godfather,Q1066512,Charles Wright,The Godfather|Papa Shango,exact-match,American professional wrestler,...,4.472805e-02,3.811774e-09,0.499339,0.000,0.00,0,0,0,0,0
4,1,0,The Godfather,1|1972|Francis Ford Coppola|1,The Godfather,Q20655440,The Godfather,NaN,exact-match,novel series,...,2.309464e-03,8.133001e-09,1.000000,0.000,0.00,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12219,1,99,Persona,100|1966|Ingmar Bergman|158,Persona,Q486329,Fergana,NaN,fuzzy-augmented,capital of Fergana Province in eastern Uzbekistan,...,1.028111e-05,0.000000e+00,0.630952,0.000,0.00,0,0,0,0,0
12220,1,99,Persona,100|1966|Ingmar Bergman|158,Persona,Q6584160,Mount Parsons,NaN,fuzzy-augmented,mountain in Antarctica,...,7.593983e-07,0.000000e+00,0.733333,0.000,0.00,0,0,0,0,0
12221,1,99,Persona,100|1966|Ingmar Bergman|158,Persona,Q16879085,Nathan Everett Pearson,Nathan E. Pearson|Pearson,fuzzy-augmented,American ichthyologist,...,2.962246e-07,0.000000e+00,0.765344,0.000,0.00,0,0,0,0,0
12222,1,99,Persona,100|1966|Ingmar Bergman|158,Persona,Q11815667,Pergola in Wroclaw,Pergola in Wrocław,fuzzy-augmented,NaN,...,4.231670e-07,0.000000e+00,0.680952,0.000,0.00,0,0,0,0,0


In [20]:
!ls $training_features_dir | wc -l

      44


In [21]:
# get training data
all_data = merge_df([f'{training_features_dir}/{fid}.csv' for fid in file_ids])
all_data.head()

,column,row,label,context,label_clean,kg_id,kg_labels,kg_aliases,method,kg_descriptions,...,aligned_pagerank,monge_elkan.1,des_cont_jaccard.1,des_cont_jaccard_normalized,vote_aligned_pagerank,vote_smallest_qnode_number,vote_monge_elkan,vote_des_cont_jaccard,votes,table_id
0,1,0,The Godfather,1|1972|Francis Ford Coppola|1,The Godfather,Q13033687,The Godfather,NaN,exact-match,Grimm,...,7.119850e-09,1.000000,0.000,0.00,0,0,1,0,1,58891288_0_1117541047012405958
1,1,0,The Godfather,1|1972|Francis Ford Coppola|1,The Godfather,Q1158135,The Godfather,NaN,exact-match,soundtrack of the 1972 crime film of the same ...,...,5.141773e-09,1.000000,0.125,0.25,0,0,1,0,1,58891288_0_1117541047012405958
2,1,0,The Godfather,1|1972|Francis Ford Coppola|1,The Godfather,Q1139696,The Godfather,The Godfather: The Game|The Godfather: Mob War...,exact-match,2006 open world action-adventure video game,...,5.155339e-09,1.000000,0.000,0.00,0,0,1,0,1,58891288_0_1117541047012405958
3,1,0,The Godfather,1|1972|Francis Ford Coppola|1,The Godfather,Q1066512,Charles Wright,The Godfather|Papa Shango,exact-match,American professional wrestler,...,3.811774e-09,0.499339,0.000,0.00,0,0,0,0,0,58891288_0_1117541047012405958
4,1,0,The Godfather,1|1972|Francis Ford Coppola|1,The Godfather,Q20655440,The Godfather,NaN,exact-match,novel series,...,8.133001e-09,1.000000,0.000,0.00,0,0,1,0,1,58891288_0_1117541047012405958


#### Prepare features for training

In [24]:
training_data = all_data.loc[:, ['aligned_pagerank', 'smallest_qnode_number', 'monge_elkan', 'des_cont_jaccard_normalized', 'evaluation_label']]
training_data.loc[training_data['evaluation_label'].astype(int) != 1, 'evaluation_label'] = -1
training_data = clean_dataset(training_data)

length of df: 774370
length of df: 774370 after dropping na
length of indices_to_keep: 774292


In [25]:
# balance the positive / negative cases
balanced_training_data = pd.DataFrame()
balanced_training_data = balanced_training_data.append(training_data[training_data['evaluation_label'] == 1])
balanced_training_data = balanced_training_data.append(training_data[training_data['evaluation_label'] != 1].sample(9500))

In [26]:
training_features = balanced_training_data.loc[:, ['aligned_pagerank', 'smallest_qnode_number', 'monge_elkan', 'des_cont_jaccard_normalized']]
training_labels = balanced_training_data.loc[:, ['evaluation_label']]

#### Train classifier using training data

In [27]:
# train model
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(class_weight={-1:0.3, 1:0.7})
# Train our classifier
model = lr.fit(training_features, training_labels)

/Users/summ7t/dev/table-linker/tl_env/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [28]:
# dump trained model to disk
import pickle

with open(trained_model_file, 'wb') as fid:
    pickle.dump(model, fid)

In [29]:
# load model from disk
with open(trained_model_file, 'rb') as fid:
    model_loaded = pickle.load(fid)

#### Test using dev dataset

In [12]:
import os

eval_file_names = []
eval_file_ids = []

for (dirpath, dirnames, filenames) in os.walk(dev_features_dir):
    for fn in filenames:
        if "csv" not in fn:
            continue
        abs_fn = dirpath + fn
        assert os.path.isfile(abs_fn)
        if os.path.getsize(abs_fn) == 0:
            continue
        eval_file_names.append(abs_fn)
        eval_file_ids.append(fn.split('.csv')[0])
len(eval_file_names), eval_file_ids[:3]

(9,
 ['39759273_0_1427898308030295194',
  '45073662_0_3179937335063201739',
  '29414811_2_4773219892816395776'])

In [13]:
# generate testing data
testing_data = merge_df(eval_file_names)
testing_data.loc[testing_data['evaluation_label'].astype(int) != 1, 'evaluation_label'] = -1
testing_data = clean_dataset(testing_data)
testing_features = testing_data.loc[:, ['aligned_pagerank', 'smallest_qnode_number', 'monge_elkan', 'des_cont_jaccard_normalized']]
testing_labels = testing_data.loc[:, ['evaluation_label']]

length of df: 101127
length of df: 101127 after dropping na
length of indices_to_keep: 101125


In [14]:
res = {}
for tid in eval_file_ids:
    test_data = testing_data[testing_data['table_id'] == tid]
    test_features = test_data.loc[:, ['aligned_pagerank', 'smallest_qnode_number', 'monge_elkan', 'des_cont_jaccard_normalized']]
    test_labels = test_data.loc[:, ['evaluation_label']]

    preds = model_loaded.predict(test_features)
    assert len(preds) ==  len(test_labels)

    prob = model_loaded.predict_proba(test_features)
    
    test_df = test_data.copy()
    test_df['pred'] = preds
    test_df['prob_0'] = [p[0] for p in prob]
    test_df['prob_1'] = [p[1] for p in prob]
    res[tid] = test_df

In [17]:
res['39759273_0_1427898308030295194']

,column,row,label,context,label_clean,kg_id,kg_labels,kg_aliases,method,kg_descriptions,...,des_cont_jaccard_normalized,vote_aligned_pagerank,vote_smallest_qnode_number,vote_monge_elkan,vote_des_cont_jaccard,votes,table_id,pred,prob_0,prob_1
0,1,0,The Social Network,1|2010|David Fincher|8.3|45993,The Social Network,Q16353709,The Social Network,NaN,exact-match,Wikimedia disambiguation page,...,0.000000,0,0,1,0,1,39759273_0_1427898308030295194,1,0.216134,0.783866
1,1,0,The Social Network,1|2010|David Fincher|8.3|45993,The Social Network,Q185888,The Social Network,Social Network,exact-match,2010 film by David Fincher,...,0.800000,0,1,1,1,3,39759273_0_1427898308030295194,1,0.000005,0.999995
2,1,0,The Social Network,1|2010|David Fincher|8.3|45993,The Social Network,Q1952928,The Social Network,NaN,exact-match,2010 soundtrack album by Trent Reznor and Atti...,...,0.222222,1,0,1,0,2,39759273_0_1427898308030295194,1,0.033429,0.966571
3,1,0,The Social Network,1|2010|David Fincher|8.3|45993,The Social Network,Q185888,The Social Network,Social Network,fuzzy-augmented,2010 film by David Fincher,...,0.800000,0,1,1,1,3,39759273_0_1427898308030295194,1,0.000005,0.999995
4,1,0,The Social Network,1|2010|David Fincher|8.3|45993,The Social Network,Q1952928,The Social Network,NaN,fuzzy-augmented,2010 soundtrack album by Trent Reznor and Atti...,...,0.222222,0,0,1,0,1,39759273_0_1427898308030295194,1,0.033429,0.966571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11988,1,99,Casablanca,100|1942|Michael Curtiz|8.8|182487,Casablanca,Q1046658,Casa Blanca,NaN,fuzzy-augmented,human settlement,...,0.000000,0,0,0,0,0,39759273_0_1427898308030295194,-1,0.695099,0.304901
11989,1,99,Casablanca,100|1942|Michael Curtiz|8.8|182487,Casablanca,Q2160914,NaN,NaN,fuzzy-augmented,NaN,...,0.000000,0,0,0,0,0,39759273_0_1427898308030295194,-1,0.999802,0.000198
11990,1,99,Casablanca,100|1942|Michael Curtiz|8.8|182487,Casablanca,Q649078,Casablanca Prefecture,Casablanca Province,fuzzy-augmented,prefecture of Morocco,...,0.000000,0,0,0,0,0,39759273_0_1427898308030295194,-1,0.551121,0.448879
11991,1,99,Casablanca,100|1942|Michael Curtiz|8.8|182487,Casablanca,Q844124,Government House of the Russian Federation,White House (Moscow),fuzzy-augmented,government building in Moscow,...,0.000000,0,0,0,0,0,39759273_0_1427898308030295194,-1,0.984839,0.015161
